<a href="https://colab.research.google.com/github/Dev-ZC/PoliticalOrientiationTextClassification/blob/main/PoliticalClassificationMistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install mistral mistralai

In [71]:
import pandas as pd
import json
import random
import os
from mistralai import Mistral
from mistralai.client import MistralClient
#from mistralai.models.fine_tuning import FineTuningJob
from sklearn.model_selection import train_test_split

In [72]:
# Load and rename columns
df = pd.read_csv("/content/new_election_data(2).csv")
df = df.rename(columns={"whisper_voice_to_text": "prompt", "manual_leaning": "label"})

# Split into train and validation (90% train, 10% val)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Function to write JSONL
def save_jsonl(df_split, filename):
    with open(filename, "w") as f:
        for _, row in df_split.iterrows():
            example = {
                "messages": [
                    {"role": "user", "content": f"Classify the political leaning in this text:\n\n{row['prompt']}"},
                    {"role": "assistant", "content": f"This text has a {row['label']} political leaning."}
                ]
            }
            f.write(json.dumps(example) + "\n")

# Save both files
save_jsonl(train_df, "train.jsonl")
save_jsonl(val_df, "val.jsonl")

In [75]:
# prompt: print the head of hte jsonl

import json

with open("train.jsonl", "r") as f:
    for i in range(5):  # Print the first 5 lines
        line = f.readline()
        if not line:
            break  # Stop if the file ends before 5 lines
        try:
            data = json.loads(line)
            print(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {i+1}: {e}")


{'messages': [{'role': 'user', 'content': "Classify the political leaning in this text:\n\n I'm just going to say it. I am not excited about the debate between Tim Walz and J.D. Vance. Not because I don't think that Tim Walz can do it. I think his message is stronger. It's more populous. It's for the people. I think his message actually makes sense. I think J.D. Vance is just weird. I'm not worried about that now. Walz can get his point across. He can pet people up. He can make people feel like he actually knows what he's talking about. Like he knows what they're going through. So I'm not worried about the messaging. Partially. This is the problem I have with it. They're not doc checking. They're letting the other candidates. The candidate. The opponent fact check. It's the job of the media to give us objective reality. To tell us what the truth is. Not what one person says and what the other person says and you can decide. That's not objective. That's what we need to have is an object

validate data

In [46]:
!apt-get install -y build-essential libssl-dev libffi-dev python3-dev
!pip install wget mistral_common mistral_finetune

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libffi-dev is already the newest version (3.4.2-4).
libssl-dev is already the newest version (3.0.2-0ubuntu1.19).
python3-dev is already the newest version (3.10.6-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [50]:
import wget
import mistral_common
import mistral_finetune
#!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py
#!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/validate_data.py
# reformat data
!python reformat_data.py mistral_finetune_dataset.jsonl
# validate data
!python validate_data.py mistral_finetune_dataset.jsonl


Traceback (most recent call last):
  File "/content/validate_data.py", line 16, in <module>
    from finetune.args import TrainArgs
ModuleNotFoundError: No module named 'finetune'


In [60]:
# prompt: create a test jsonl file using the following: {
#     "messages": [
#         {
#             "role": "user",
#             "content": "User interaction n°1 contained in document n°2"
#         },
#         {
#             "role": "assistant",
#             "content": "Bot interaction n°1 contained in document n°2"
#         },
#         {
#             "role": "user",
#             "content": "User interaction n°2 contained in document n°1"
#         },
#         {
#             "role": "assistant",
#             "content": "Bot interaction n°2 contained in document n°1"
#         }
#     ]
# }

import json

data = {
    "messages": [
        {
            "role": "user",
            "content": "User interaction n°1 contained in document n°2"
        },
        {
            "role": "assistant",
            "content": "Bot interaction n°1 contained in document n°2"
        },
        {
            "role": "user",
            "content": "User interaction n°2 contained in document n°1"
        },
        {
            "role": "assistant",
            "content": "Bot interaction n°2 contained in document n°1"
        }
    ]
}

with open("test.jsonl", "w") as f:
    json.dump(data, f)
    f.write('\n') # jsonl requires a newline character after each json object



Launch Training Job

In [76]:
from google.colab import userdata
API_KEY = userdata.get('MISTRAL_KEY')

# Set up Mistral client with API key
#api_key = os.environ.get("MISTRAL_API_KEY", "your_api_key_here")
client = Mistral(api_key=API_KEY)

# Upload training file
training_data = client.files.upload(
    file={
        #"file_name": "mistral_finetune_dataset.jsonl",
        #"content": open("mistral_finetune_dataset.jsonl", "rb"),
        "file_name": "train.jsonl",
        "content": open("/content/train.jsonl", "rb"),
    }
)

validation_data = client.files.upload(
    file={
        #"file_name": "mistral_finetune_dataset.jsonl",
        #"content": open("mistral_finetune_dataset.jsonl", "rb"),
        "file_name": "val.jsonl",
        "content": open("/content/val.jsonl", "rb"),
    }
)

# create a fine-tuning job
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": training_data.id, "weight": 1}],
    validation_files=[validation_data.id],
    hyperparameters={
        "training_steps": 10,
        "learning_rate":0.0001
    },
    auto_start=False
)

Check status

In [77]:
client.fine_tuning.jobs.get(job_id = created_jobs.id)

DetailedJobOut(id='baa1ff71-3a6c-4f96-828f-52b6bee7fd19', auto_start=False, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=41.24354940213971, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='VALIDATED', job_type='completion', created_at=1744662846, modified_at=1744662852, training_files=['db68a0fb-c075-49a7-b706-75fe9c96d80f'], validation_files=['010ccef1-9277-4a86-ac25-1d6cd4f30760'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=250, cost=2.63, cost_currency='USD', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=31780, estimated_start_time=None), events=[EventOut(name='status-updated', created_at=1744662846, data={'status': 'QUEUED'}), EventOut(name='status-updated', created_at=1744662847, data={'status': 'VALIDATING'}), EventOut(name='status-updated', created_at=1744662

In [78]:
# start a fine-tuning job
client.fine_tuning.jobs.start(job_id = created_jobs.id)

created_jobs

JobOut(id='baa1ff71-3a6c-4f96-828f-52b6bee7fd19', auto_start=False, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='completion', created_at=1744662846, modified_at=1744662846, training_files=['db68a0fb-c075-49a7-b706-75fe9c96d80f'], validation_files=['010ccef1-9277-4a86-ac25-1d6cd4f30760'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

Check finetuneing status

In [92]:
from google.colab import userdata
API_KEY = userdata.get('MISTRAL_KEY')
import datetime

# Set up Mistral client with API key
#api_key = os.environ.get("MISTRAL_API_KEY", "your_api_key_here")
client = Mistral(api_key=API_KEY)

# List jobs
jobs = client.fine_tuning.jobs.list()
print(jobs)

# Retrieve a jobs
#retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
#print(retrieved_jobs)

# Cancel a jobs
#canceled_jobs = client.fine_tuning.jobs.cancel(job_id = created_jobs.id)
#print(canceled_jobs)

total=7 data=[JobOut(id='baa1ff71-3a6c-4f96-828f-52b6bee7fd19', auto_start=False, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=41.24354940213971, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='SUCCESS', job_type='completion', created_at=1744662846, modified_at=1744663193, training_files=['db68a0fb-c075-49a7-b706-75fe9c96d80f'], validation_files=['010ccef1-9277-4a86-ac25-1d6cd4f30760'], OBJECT='job', fine_tuned_model='ft:open-mistral-7b:86677200:20250414:baa1ff71', suffix=None, integrations=[], trained_tokens=1310720, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=250, cost=2.63, cost_currency='USD', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=31780, estimated_start_time=None)), JobOut(id='27618905-f91c-498e-87ba-4a2081cb83eb', auto_start=False, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction

Use model

In [99]:
from google.colab import userdata
API_KEY = userdata.get('MISTRAL_KEY')

# Set up Mistral client with API key
#api_key = os.environ.get("MISTRAL_API_KEY", "your_api_key_here")
client = Mistral(api_key=API_KEY)

retrieved_model = client.fine_tuning.jobs.get(job_id = "baa1ff71-3a6c-4f96-828f-52b6bee7fd19")

prompt = "Classify the political leaning of this text, output must be single word (either Republican, Democrat, or Neither):"

text_input = "I am a republican"

chat_response = client.chat.complete(
    model=retrieved_model.fine_tuned_model,
    messages = [{"role":'user', "content":f'{prompt} {text_input}'}]
)

print(chat_response.choices[0].message)

content='Republican' tool_calls=None prefix=False role='assistant'
